### Initial Setup

The following cells install dependencies and set up LLM tracing.


In [1]:
# load required env vars for ddtrace
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
import ddtrace.auto

ddtrace.patch_all()

from ddtrace.llmobs import LLMObs

LLMObs.enable()

### Tracing an LLM Span

An LLM Span represents a call to a model. In this example, we are asking `gpt-3.5-turbo` to summarize a provided text and identify a list of topics from the text.

Because we use OpenAI, the call to the LLM is instrumented automatically:


In [3]:
from openai import OpenAI
import os

oai_client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))

sys_prompt = """
Your task is to 
1. Summarize the given text at a 6th grade reading level in no more than 2 sentences.
2. Identify what topics the text belongs to that would allow you to categorize it in a school library.
Format your output strictly following this JSON convention:
{	
    "topics": <[insert array of topics here]>
    "summary": <insert summary here>
}
	"""


def summarize(text, prompt=sys_prompt):
    messages = [
        {"role": "system", "content": sys_prompt},
        {"role": "user", "content": text},
    ]
    # llm span auto-instrumented via our openai integration
    return (
        oai_client.chat.completions.create(
            messages=messages,
            model="gpt-3.5-turbo",
            response_format={"type": "json_object"},
        )
        .choices[0]
        .message.content
    )

In [4]:
text = """
ONE JANUARY day, thirty years ago, the little town of Hanover, anchored on a windy Nebraska tableland, was trying not to be blown away. A mist of fine snowflakes was curling and eddying about the cluster of low drab buildings huddled on the gray prairie, under a gray sky. The dwelling-houses were set about haphazard on the tough prairie sod; some of them looked as if they had been moved in overnight, and others as if they were straying off by themselves, headed straight for the open plain. None of them had any appearance of permanence, and the howling wind blew under them as well as over them. The main street was a deeply rutted road, now frozen hard, which ran from the squat red railway station and the grain "elevator" at the north end of the town to the lumber yard and the horse pond at the south end. On either side of this road straggled two uneven rows of wooden buildings; the general merchandise stores, the two banks, the drug store, the feed store, the saloon, the post-office. The board sidewalks were gray with trampled snow, but at two o'clock in the afternoon the shopkeepers, having come back from dinner, were keeping well behind their frosty windows.
"""

In [5]:
import json

# print out the LLM's response:
json.loads(summarize(text))

Error submitting packet: [Errno 61] Connection refused, dropping the packet and closing the socket


{'topics': ['Historical Fiction',
  'Midwest America',
  'Small Town',
  'Winter Weather',
  'Rural Life'],
 'summary': "In a small Nebraskan town thirty years ago, the harsh winter weather and winds threatened to blow away the cluster of buildings on the prairie. The town's main street, lined with various stores, was deserted as shopkeepers stayed indoors to escape the freezing temperatures."}

Now, try checking out the LLM Observability interace in Datadog. You should see a trace that describes the LLM call, including the system prompt, the user prompt, and the response.


In [7]:
## TODO: submit a new call that triggers a quality metric alert